### Sequence-to-sequence example in Keras for Translation (character-level)


#### Algorithm:
&#9673; Use Dataset with pairs of translated sentences <br>
&#9673; An encoder LSTM turns input sequences to 2 state vectors (keep the last LSTM state and discard the outputs). <br>
&#9673; A decoder LSTM is trained to turn the target sequences into the same sequence but offset by one timestep in the future, a training process called "teacher forcing" in this context. The decoder learns to generate `targets[t+1...]` given `targets[...t]`, conditioned on the input sequence. <br>

In inference mode, to decode unknown input sequences: <br>
&#9673; Encode the input sequence into state vectors <br>
&#9673; Start with a target sequence of size 1 (just the start-of-sequence character) <br>
&#9673; Feed the state vectors and 1-char target sequence to the decoder to produce predictions for the next character <br>
&#9673; Sample the next character using these predictions (we simply use argmax) <br>
&#9673; Append the sampled character to the target sequence <br>
&#9673; Repeat until we generate the end-of-sequence character or we hit the character limit <br>

#### Source:
&#9673; Dataset:  Hindi - English (hin-eng.zip) (2808) (http://www.manythings.org/anki/_ <br>
&#9673; Source 1: https://www.analyticsvidhya.com/blog/2018/03/essentials-of-deep-learning-sequence-to-sequence-modelling-with-attention-part-i/ <br>
&#9673; Source 2: https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py <br>

In [11]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

# Path to the data txt file on disk.
data_path = 'Datasets/Translation-Datasets/English-Hindi/hin.txt'

In [3]:
# Tokenise the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]: #choose atleast 10,000 samples to train on
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 2808
Number of unique input tokens: 70
Number of unique output tokens: 92
Max sequence length for inputs: 124
Max sequence length for outputs: 123


In [4]:
# Vectorise the data
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [5]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [6]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [7]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [8]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model 
model.save('s2s.h5')
print("Saved model weights to disk")



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 2246 samples, validate on 562 samples
Epoch 1/100
2246/2246 [==============================] - 39s 18ms/step - loss: 1.1696 - acc: 0.7851 - val_loss: 1.8878 - val_acc: 0.6881
Epoch 2/100
2246/2246 [==============================] - 39s 18ms/step - loss: 0.8620 - acc: 0.8085 - val_loss: 1.4230 - val_acc: 0.6877
Epoch 3/100
2246/2246 [==============================] - 44s 19ms/step - loss: 0.8359 - acc: 0.8097 - val_loss: 1.5982 - val_acc: 0.6885
Epoch 4/100
2246/2246 [==============================] - 46s 20ms/step - loss: 0.7826 - acc: 0.8119 - val_loss: 1.4385 - val_acc: 0.6883
Epoch 5/100
2246/2246 [==============================] - 46s 21ms/step - loss: 0.7256 - acc: 0.8208 - val_loss: 1.2058 - val_acc: 0.6972
Epoch 6/100
2246/2246 [==============================] - 47s 21ms/step - loss: 0.6743 - acc: 0.8345 - val_loss: 1.1073 - val_acc: 0.7218
Epoch 7/100
2246/2246 [===========

Epoch 53/100
2246/2246 [==============================] - 47s 21ms/step - loss: 0.2894 - acc: 0.9181 - val_loss: 0.8034 - val_acc: 0.7899
Epoch 54/100
2246/2246 [==============================] - 47s 21ms/step - loss: 0.2859 - acc: 0.9194 - val_loss: 0.8175 - val_acc: 0.7879
Epoch 55/100
2246/2246 [==============================] - 47s 21ms/step - loss: 0.2821 - acc: 0.9204 - val_loss: 0.8182 - val_acc: 0.7871
Epoch 56/100
2246/2246 [==============================] - 47s 21ms/step - loss: 0.2779 - acc: 0.9217 - val_loss: 0.7997 - val_acc: 0.7919
Epoch 57/100
2246/2246 [==============================] - 48s 21ms/step - loss: 0.2738 - acc: 0.9229 - val_loss: 0.8299 - val_acc: 0.7888
Epoch 58/100
2246/2246 [==============================] - 47s 21ms/step - loss: 0.2695 - acc: 0.9238 - val_loss: 0.8194 - val_acc: 0.7888
Epoch 59/100
2246/2246 [==============================] - 48s 21ms/step - loss: 0.2652 - acc: 0.9253 - val_loss: 0.8285 - val_acc: 0.7861
Epoch 60/100
2246/2246 [==========

In [9]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model( 
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [10]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Wow!
Decoded sentence: तुम कैसे हो?

-
Input sentence: Help!
Decoded sentence: तुम्हे इस लिए टेट नहीं के तेज़ा कान करना चाहिए।

-
Input sentence: Jump.
Decoded sentence: वह अंदर ले जाने का पना था।

-
Input sentence: Jump.
Decoded sentence: वह अंदर ले जाने का पना था।

-
Input sentence: Jump.
Decoded sentence: वह अंदर ले जाने का पना था।

-
Input sentence: Hello!
Decoded sentence: तुम्हे इस लिए टेट बहने चाए गें में देख रहीं हैं।

-
Input sentence: Hello!
Decoded sentence: तुम्हे इस लिए टेट बहने चाए गें में देख रहीं हैं।

-
Input sentence: Cheers!
Decoded sentence: तुम कैसे हो?

-
Input sentence: Cheers!
Decoded sentence: तुम कैसे हो?

-
Input sentence: Got it?
Decoded sentence: तुम कितने बजे उसके उसके बारे में कप छाता था।

-
Input sentence: I'm OK.
Decoded sentence: मेरे पास के लिए सेख बहुए थे जा नहीं जाता है।

-
Input sentence: Awesome!
Decoded sentence: तुम्हे इस लिए टेट बहने चाए गें में देख रहीं हैं।

-
Input sentence: Come in.
Decoded sentence: तुम्हे इस लिए टे लिए ब

-
Input sentence: Happy Easter!
Decoded sentence: तुम्हे इस लिए टेट बहने चाए गें में देख रहीं हैं।

-
Input sentence: Has Tom left?
Decoded sentence: तुम्हे इस लिए टेट नहीं के तेज़ा कान करना चाहिए।

-
Input sentence: He is French.
Decoded sentence: वह अंदर ले जाने के पार जाने-का हुश था।

-
Input sentence: I am at home.
Decoded sentence: मेरे पास के लिए सेख बहुए थे जा नहीं जाता है।

-
Input sentence: I can't move.
Decoded sentence: मेरे पास के लिए सेख बहुए थे जा नहीं जाता है।

